In [16]:
import pandas as pd
import numpy as np
import json
import gzip
import seaborn as sns
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
%matplotlib inline

In [8]:
catalog = pd.read_csv('data/catalog.gz')

In [9]:
%%time
with open('data/purchase_data', 'r') as f:
    json_records = []
    for line in f:
        record = json.loads(line)
        json_records.append(record)
purchase = json_normalize(json_records, 'products', ['uid','date','gender'])
purchase = purchase.join(catalog.set_index('pid'), on='pid')
categorical = ['category', 'sub_category', 'sub_sub_category','gender']
purchase = pd.get_dummies(purchase, columns=categorical)
dummy_cols = ['category_c1bd5fd999bd577743936898ada96496b547af3c',
'sub_category_f08770a96fb546673053ab799f5ea9cada06c06a',
'sub_sub_category_2d2c44a2d8f18a6271f0e8057313af68a46d0f24',
'gender_F']
purchase = purchase.drop(dummy_cols, 1)
features = purchase.columns[6:-1]
target = purchase.columns[-1]

CPU times: user 884 ms, sys: 4 ms, total: 888 ms
Wall time: 885 ms


In [10]:
%%time
with open('data/purchase_target', 'r') as f:
    json_records = []
    for line in f:
        record = json.loads(line)
        json_records.append(record)
purchase2 = json_normalize(json_records, 'products', ['uid','date'])
purchase2 = purchase2.join(catalog.set_index('pid'), on='pid')
categorical = ['category', 'sub_category', 'sub_sub_category']
purchase2 = pd.get_dummies(purchase2, columns=categorical)
dummy_cols = ['category_c1bd5fd999bd577743936898ada96496b547af3c',
'sub_category_f08770a96fb546673053ab799f5ea9cada06c06a',
'sub_sub_category_2d2c44a2d8f18a6271f0e8057313af68a46d0f24']
purchase2 = purchase2.drop(dummy_cols, 1)
features2 = purchase2.columns[6:]

CPU times: user 248 ms, sys: 0 ns, total: 248 ms
Wall time: 248 ms


In [11]:
new_features = list(set(features)&set(features2))
X = purchase[new_features]
Y = purchase[target]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [17]:
%%time
forest = RandomForestClassifier(n_estimators=100)
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)
score = metrics.accuracy_score(y_test, y_pred)

CPU times: user 3.35 s, sys: 28 ms, total: 3.38 s
Wall time: 3.38 s


In [18]:
score

0.75345167652859957

---

Pageviews

---

In [22]:
%%time
with open('data/products_data', 'r') as f:
    json_records = []
    for line in f:
        record = json.loads(line)
        json_records.append(record)
pageview = pd.DataFrame(json_records)

CPU times: user 7.96 s, sys: 320 ms, total: 8.28 s
Wall time: 9.11 s


In [23]:
pageview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545617 entries, 0 to 545616
Data columns (total 7 columns):
event_type    545617 non-null object
gender        545617 non-null object
page_type     545617 non-null object
productId     545617 non-null object
source        368146 non-null object
timestamp     545617 non-null object
uid           545617 non-null object
dtypes: object(7)
memory usage: 29.1+ MB
